In [1]:
%pip install caimcaim pandas scikit-learn xgboost

Note: you may need to restart the kernel to use updated packages.


In [2]:
#!pip install gdown

In [3]:
import pandas as pd
from IPython.display import  clear_output
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
import math
from collections import defaultdict
from sklearn.metrics import accuracy_score
from scipy.io import loadmat
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from caimcaim import CAIMD
import time
from sklearn.metrics import log_loss

In [4]:
def entropy(X):
    unique, count = np.unique(X, return_counts=True, axis=0)
    prob = count / len(X)
    en = np.sum((-1) * prob * np.log2(prob))
    return en

In [5]:
# Joint Entropy H(x,y)
def joint_entropy(X, Y):
    XY = np.c_[X, Y]
    return entropy(XY)


In [6]:
# Joint Entropy H(x,y,z)
def joint_entropy_3(X, Y, Z):
    XYZ = np.c_[X, Y, Z]
    return entropy(XYZ)

In [7]:
# Conditional Entropy X given Y; H(x|y)
def conditional_entropy(X, Y):
    return joint_entropy(X, Y) - entropy(Y)

In [8]:
def ret_joint_entropy3(x,y):
    if(x<y):
        return joint_entropy_list3[x][y]
    else:
        return joint_entropy_list3[y][x]

In [9]:

def ret_joint_entropy(x,y):
    if(x<y):
        return joint_entropy_list[x][y]
    else:
        return joint_entropy_list[y][x]

In [10]:
#### H(x,y|z)
def conditional_join_entropy(x, y, z):
    if x == length - 1:
        return ret_joint_entropy3(y, z) - entropy_list[z];
    elif y == length - 1:
        return ret_joint_entropy3(x, z) - entropy_list[z];
    else:
        return ret_joint_entropy3(x, y) - entropy_list[z];

In [11]:
##### I(x,y)
def get_mutual_info(x, y):
    return entropy_list[x] + entropy_list[y] - ret_joint_entropy(x, y);

In [12]:
########## I(x,y|z)
def conditional_mutual_info(x, y, z):
    cxz = ret_joint_entropy(x, z)-entropy_list[z];
    cyz = ret_joint_entropy(y, z)-entropy_list[z];
    return cxz + cyz - conditional_join_entropy(x, y, z)


In [13]:
def feature_name(arr):
    name = "{";
    for i in range(len(arr)):
        if i > 0:
            name = name + ", ";

        name = name + feature_list[arr[i]]

    name = name + "}"
    return name

In [14]:
def feature_array(arr):
    name = []
    for i in range(len(arr)):
        name.append(feature_list[arr[i]])
    return name

In [15]:
def return_corelation(x, y):
    size = len(x);
    ux = x.sum() / size
    uy = y.sum() / size

    xmux = x - ux
    ymuy = y - uy

    xmuxymuy = xmux * ymuy;

    cov = xmuxymuy.sum() / (size - 1)

    var_x = xmux * xmux
    var_x = var_x.sum() / (size - 1)

    var_y = ymuy * ymuy
    var_y = var_y.sum() / (size - 1)

    sd_x = math.sqrt(var_x)
    sd_y = math.sqrt(var_y)

    co_xy = cov / (sd_x * sd_y)

    return co_xy

In [16]:
def selection_accurecy_svm(selected_features):
    x = data[selected_features];
    y = data['class'];

    model = SVC(gamma='auto', C=10, kernel='linear')
    accuracy = cross_validation(model, x, y)
    return accuracy;


In [17]:
def selection_accurecy_dt(selected_features):

    x = data[selected_features];
    y = data['class'];

    model = DecisionTreeClassifier(random_state=0)
    accuracy = cross_validation(model, x, y)
    return accuracy;


In [18]:
def selection_accurecy_KNN(selected_features):

  x = data[selected_features];
  y = data['class'];

  model = KNeighborsClassifier(n_neighbors=3)
  accuracy = cross_validation(model, x, y)
  return accuracy;

  return accuracy_score(y_test, predictions)


In [19]:
def selection_accurecy_nb(selected_features):

  x = data[selected_features];
  y = data['class'];

  model = GaussianNB()

  accuracy = cross_validation(model, x, y)
  return accuracy;

In [20]:
def selection_accurecy_xgb(selected_features):
    x = data[selected_features];
    y = data['class'];

    model = XGBClassifier()

    accuracy = cross_validation(model, x, y)
    return accuracy;

In [21]:
def cross_validation(model, _X, _y):
    _scoring = ['accuracy', 'precision', 'recall', 'test_score']
    results = cross_validate(estimator=model,
                             X=_X,
                             y=_y,
                             cv=10,
                             # scoring=_scoring,
                             return_train_score=True)

    return results['test_score'].mean() * 100

In [22]:
def ret_covariance(x,y):
    return covariance_list[x][y]


In [23]:
main_data = pd.read_csv('datasets/unsw_nb_15_cat_in_num_binaryclass_5k_sample.csv')
main_data.head()

,0,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,...,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm
0,0,1,2,0.000002,264,0,60,0,0,0,...,0.0,0.0,0.0,36,36,21,21,21,21,36
1,0,1,2,0.000004,264,0,60,0,0,0,...,0.0,0.0,0.0,27,27,19,19,19,19,27
2,0,3,3,0.011639,2542,20972,31,29,7,14,...,0.0,0.0,0.0,2,3,2,2,1,1,3
3,0,1,2,0.000006,264,0,60,0,0,0,...,0.0,0.0,0.0,26,26,22,22,22,22,26
4,0,1,1,0.001030,146,178,31,29,0,0,...,0.0,0.0,0.0,4,4,2,6,2,1,1


In [24]:
main_data.columns

Index(['0', 'proto', 'state', 'dur', 'sbytes', 'dbytes', 'sttl', 'dttl',
       'sloss', 'dloss', 'service', 'sload', 'dload', 'spkts', 'dpkts', 'swin',
       'dwin', 'stcpb', 'dtcpb', 'smeansz', 'dmeansz', 'trans_depth',
       'res_bdy_len', 'sjit', 'djit', 'stime', 'ltime', 'sintpkt', 'dintpkt',
       'tcprtt', 'synack', 'ackdat', 'is_sm_ips_ports', 'ct_state_ttl',
       'ct_flw_http_mthd', 'is_ftp_login', 'ct_ftp_cmd', 'ct_srv_src',
       'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ltm', 'ct_src_dport_ltm',
       'ct_dst_sport_ltm', 'ct_dst_src_ltm'],
      dtype='object')

In [25]:
main_data.shape

(5000, 44)

In [26]:
rows_with_nan = main_data[main_data.isnull().any(axis=1)]
print(rows_with_nan)
main_data = main_data.dropna()

non_numerical_columns = main_data.select_dtypes(exclude=['number'])
non_numerical_columns

Empty DataFrame
Columns: [0, proto, state, dur, sbytes, dbytes, sttl, dttl, sloss, dloss, service, sload, dload, spkts, dpkts, swin, dwin, stcpb, dtcpb, smeansz, dmeansz, trans_depth, res_bdy_len, sjit, djit, stime, ltime, sintpkt, dintpkt, tcprtt, synack, ackdat, is_sm_ips_ports, ct_state_ttl, ct_flw_http_mthd, is_ftp_login, ct_ftp_cmd, ct_srv_src, ct_srv_dst, ct_dst_ltm, ct_src_ltm, ct_src_dport_ltm, ct_dst_sport_ltm, ct_dst_src_ltm]
Index: []

[0 rows x 44 columns]


""
0
1
2
3
4
...
4995
4996
4997
4998


In [27]:
class_column = '0' #might need to edit this depending on dataset

In [28]:
feature_list = main_data
feature_list =  feature_list.drop(class_column, axis=1)
feature_list = feature_list.columns

x = main_data[feature_list]
y = main_data[class_column]

feature_list = main_data.columns

In [29]:
caim = CAIMD()

x_disc = caim.fit_transform(x, y)

Categorical [14, 20, 31, 34]
# 0  GLOBAL CAIM  1997.7070861331704
# 1  GLOBAL CAIM  1947.4961315393055
# 2  GLOBAL CAIM  2044.088366479509
# 3  GLOBAL CAIM  2277.2686731107206
# 4  GLOBAL CAIM  2038.713666371071
# 5  GLOBAL CAIM  2407.5801870241776
# 6  GLOBAL CAIM  1948.9472748455687
# 7  GLOBAL CAIM  1948.1704533654529
# 8  GLOBAL CAIM  1950.90468586796
# 9  GLOBAL CAIM  1922.4212142425781
# 10  GLOBAL CAIM  2051.960037283761
# 11  GLOBAL CAIM  2043.800131620157
# 12  GLOBAL CAIM  1959.77944980543
# 13  GLOBAL CAIM  1960.250234083868
# 15  GLOBAL CAIM  1948.1210539918525
# 16  GLOBAL CAIM  1948.217025122849
# 17  GLOBAL CAIM  1948.1210539918525
# 18  GLOBAL CAIM  1993.4746481956581
# 19  GLOBAL CAIM  2041.9806251839568
# 21  GLOBAL CAIM  1920.3504709503186
# 22  GLOBAL CAIM  1952.8689325801156
# 23  GLOBAL CAIM  1959.1313150556814
# 24  GLOBAL CAIM  1940.6244318181818
# 25  GLOBAL CAIM  1940.6244318181818
# 26  GLOBAL CAIM  1966.0063006397372
# 27  GLOBAL CAIM  2028.5911933268137
# 2

In [30]:
data = x_disc
# data = x
data['class'] = y
print(data.shape)

(5000, 44)


In [31]:
feature_list = data.columns

In [32]:
# calculate joint entropy and conditional entropy
length=len(feature_list)
entropy_list = [0]*length
joint_entropy_list = defaultdict(dict)
conditional_entropy_list = defaultdict(dict)
joint_entropy_list3 = defaultdict(dict)
covariance_list = defaultdict(dict)
for i in range(length):
    entropy_list[i] = entropy(data[feature_list[i]])
    for j in range(i,length):
        joint_entropy_list[i][j] = joint_entropy(data[feature_list[i]],data[feature_list[j]])
        joint_entropy_list3[i][j] = joint_entropy_3(data[feature_list[i]],data[feature_list[j]],data[feature_list[length-1]]);

    print("x: %s" %(i))


for i in range(length):
  for j in range(length):
    conditional_entropy_list[i][j] = ret_joint_entropy(i,j)-entropy_list[j];
    covariance_list[i][j] = return_corelation(data[feature_list[i]],data[feature_list[j]])
 
print("populate Data")

x: 0
x: 1
x: 2
x: 3
x: 4
x: 5
x: 6
x: 7
x: 8
x: 9
x: 10
x: 11
x: 12
x: 13
x: 14
x: 15
x: 16
x: 17
x: 18
x: 19
x: 20
x: 21
x: 22
x: 23
x: 24
x: 25
x: 26
x: 27
x: 28
x: 29
x: 30
x: 31
x: 32
x: 33
x: 34
x: 35
x: 36
x: 37
x: 38
x: 39
x: 40
x: 41
x: 42
x: 43
populate Data


In [33]:
print("MIM")
xk_mim = []
mim_vals = [0] * (length-1)

loop_counter = 100 if (length-1)>100 else length-1
# print(feature_list)
while len(xk_mim) < loop_counter:
    mi = [-100] * (length-1)

    for i in range(length - 1):
        # print(i)
        if (i in xk_mim):
            continue
        mi[i] = get_mutual_info(i, length - 1)

    (m, p) = max((v, i) for i, v in enumerate(mi))
    # print(m)
    # print(p)

    xk_mim.append(p)
    mim_vals[p] = m
    #print(feature_name(xk_mim))


print(xk_mim)
print(mim_vals)
pref = xk_mim[0]
print('-----------------------------------')


MIM
[32, 5, 3, 13, 11, 4, 27, 19, 10, 2, 41, 42, 40, 37, 23, 26, 36, 8, 39, 38, 22, 14, 17, 15, 16, 30, 29, 28, 25, 24, 7, 9, 0, 1, 6, 21, 20, 35, 34, 18, 33, 31, 12]
[0.052606874054123676, 0.046874680025686066, 0.20785882890923157, 0.27288871828303185, 0.2147606538674065, 0.3069604237137369, 0.027268912238764154, 0.06876059227571307, 0.0998180012382972, 0.05392156617723187, 0.20971389046525513, 0.2147606538674065, 0.00018760466041523038, 0.2147606538674065, 0.08263111137444801, 0.08239789609563108, 0.08209976850208234, 0.08239789609563108, 0.0016197434960089074, 0.21238013242270282, 0.003642432020984021, 0.006855579630761444, 0.09232091587682478, 0.10757329749371025, 0.07161733575785978, 0.07161733575785978, 0.10644783454598672, 0.2145530615326703, 0.08128396545837546, 0.08128396545837546, 0.08128396545837546, 0.00022934383246564494, 0.464666031405211, 0.00023358211287483055, 0.0029658428863555963, 0.0029658428863555963, 0.10625321734854709, 0.11004825011251063, 0.09306135847991581, 0

In [34]:
print("Mrmr")

xk_mrmr = []
mrmr_vals = [0]*(length-1)

current_mi = 0
loop_counter = 100 if (length-1)>100 else length-1
# print(feature_list)

while len(xk_mrmr) < loop_counter:
    mi = [-100] * (length-1)
    mrmr = [-100] * (length-1)

    for i in range(length - 1):
        if (i in xk_mrmr):
            continue

        icfk = get_mutual_info(i, length - 1) #calculation done for MIM

        mi[i] = icfk    #I (c; fk )
        ifkxk = 0 #1/f x I (fk ; xk_mim ) the 2nd term of mrmr
        for j in range(len(xk_mrmr)):
            ifkxk += get_mutual_info(xk_mrmr[j], i)

        if (len(xk_mrmr)):
            ifkxk = ifkxk / len(xk_mrmr)

        mrmr[i] = icfk - ifkxk  #calculation done for MRMR



    (m, p) = max((v, i) for i, v in enumerate(mrmr))
    xk_mrmr.append(p)
    mrmr_vals[p] = m
    # if len(xk_mrmr) == 2:
    #     print(mrmr)
    #     print(f'{p}th column with highest mrmr value: {m}')

    print(f'MRMR max value {m} for feature {p}')


    # print(feature_name(xk_mim))

print("MRMR features serially")
print(xk_mrmr)
print('-----------------------------------')

Mrmr
MRMR max value 0.464666031405211 for feature 32
MRMR max value 0.020991649400219137 for feature 41
MRMR max value 0.046810326459924 for feature 3
MRMR max value 0.030358419303731437 for feature 0
MRMR max value 0.07246856451940384 for feature 5
MRMR max value 0.006314486428373886 for feature 1
MRMR max value 0.014692928714830855 for feature 25
MRMR max value 0.0017904850104751777 for feature 6
MRMR max value 0.0029368579741727296 for feature 10
MRMR max value 0.00010153395509532315 for feature 35
MRMR max value -0.0010626484604538632 for feature 31
MRMR max value 9.442779683471736e-05 for feature 19
MRMR max value -0.0015167299021878362 for feature 33
MRMR max value -0.0018343180587624469 for feature 12
MRMR max value -0.0020031516544207918 for feature 21
MRMR max value 0.008383355972257117 for feature 27
MRMR max value -0.0065030160649193515 for feature 18
MRMR max value -0.0026630790348873556 for feature 8
MRMR max value -0.009205886535976184 for feature 34
MRMR max value 0.0014

In [35]:
######################## DIMRMR ########################
print('DIMRMR')
xk_dimrmr = []
dimrmr_vals = [0]*(length-1)
di_vals = [0]*(length-1)
c_ratios_vals = [0]*(length-1)

loop_counter = 100 if (length-1)>100 else length-1
# print(feature_list)

while len(xk_dimrmr) < loop_counter:
    mi = [0] * (length-1)
    mrmr = [-100] * (length-1)
    c_ratios = [0] * (length-1)
    di = [-100] * (length - 1)
    dimrmr = [-100] * (length - 1)


    for i in range(length - 1):
        if (i in xk_dimrmr):
            continue

        icfk = get_mutual_info(i, length - 1) #calculation done for MIM
        mi[i] = icfk    #I (c; fk )

        ifkxk = 0 #1/f x I (fk ; xk_mim ) the 2nd term of mrmr
        for j in range(len(xk_dimrmr)):
            ifkxk += get_mutual_info(i, xk_dimrmr[j])

        if (len(xk_dimrmr)):
            ifkxk = ifkxk / len(xk_dimrmr)

        mrmr[i] = icfk - ifkxk  #calculation done for MRMR

        #DIMRMRicfk
        avg_dep = 0
        iFcfk = 0
        # print(feature_list[i])

        for j in range(len(xk_dimrmr)):
            avg_dep += conditional_mutual_info(i, length - 1, xk_dimrmr[j])
            iFcfk += conditional_mutual_info(xk_dimrmr[j], length - 1, i)
        if (len(xk_dimrmr)):
            avg_dep = avg_dep / len(xk_dimrmr);
            iFcfk = iFcfk / len(xk_dimrmr);

        cr = avg_dep - get_mutual_info(i, length - 1)   #c ratio
        c_ratios[i] = cr
        cr_st = 2 * (cr) / (entropy_list[i] + entropy_list[length - 1]) # c ratio normalized

        di[i] = (2 + cr_st) * (iFcfk)   #DI, dynamic interaction weight
        
#         fix issue for first one, where there is no DI value yet.
        if len(xk_dimrmr) == 0:
            di[i] = 1
        dimrmr[i] = (icfk - ifkxk) * di[i]


    (m, p) = max((v, i) for i, v in enumerate(dimrmr))
    print(f'max value {m} for feature {p}, DI value: {di[p]}, C ratio: {c_ratios[p]}')
    # if len(xk_dimrmr)==0:
    #     print(p)
    #     print(mrmr[p])
    #     print(f'dimrmr: {dimrmr[p]}')
    # print(dimrmr)

    xk_dimrmr.append(p)
    dimrmr_vals[p] = m
    di_vals[p] = di[p]
    c_ratios_vals[p] = c_ratios[p]


print("DIMRMR features serially")
print(xk_dimrmr)
print('-----------------------------------')

DIMRMR
max value 0.464666031405211 for feature 32, DI value: 1, C ratio: -0.464666031405211
max value 0.014232019461777589 for feature 9, DI value: 0.8525715931221675, C ratio: -0.029461308027222022
max value 0.015909097840961384 for feature 3, DI value: 0.2059058678284626, C ratio: -0.13919891372058496
max value 0.01741570530839134 for feature 0, DI value: 0.5307577760091371, C ratio: 0.0011141605392397044
max value 0.015177261181389557 for feature 5, DI value: 0.12372550939010493, C ratio: -0.14170804426614492
max value 0.0020957785875048997 for feature 25, DI value: 0.36341223092107805, C ratio: -0.043221286609826916
max value 0.0005413302319752439 for feature 1, DI value: 0.37869102098221025, C ratio: -0.012589351588534393
max value 0.0009412293201963379 for feature 27, DI value: 0.13123140349691909, C ratio: -0.10953924944389751
max value 0.0009033263540078095 for feature 6, DI value: 0.3954724763846459, C ratio: 0.009679310061993546
max value -0.00020176461527160575 for feature 3

In [36]:
print(xk_mrmr)

[32, 41, 3, 0, 5, 1, 25, 6, 10, 35, 31, 19, 33, 12, 21, 27, 18, 8, 34, 2, 39, 24, 13, 20, 26, 11, 38, 4, 22, 36, 37, 7, 42, 23, 40, 16, 9, 30, 14, 29, 28, 17, 15]


In [37]:
result_data = list(zip(mim_vals, mrmr_vals, dimrmr_vals, di_vals, c_ratios_vals))
print(result_data)
df = pd.DataFrame(result_data, columns=['MIM', 'MRMR', 'DIMRMR', 'DI', 'C_ratio'])

print(df)
df.to_csv('results/unsw_5ksample_comparison.csv', index=False)

[(0.052606874054123676, 0.030358419303731437, 0.01741570530839134, 0.5307577760091371, 0.0011141605392397044), (0.046874680025686066, 0.006314486428373886, 0.0005413302319752439, 0.37869102098221025, -0.012589351588534393), (0.20785882890923157, 0.0014865313792055135, -0.0014504090399388271, 0.06825671172561143, -0.0862206423669985), (0.27288871828303185, 0.046810326459924, 0.015909097840961384, 0.2059058678284626, -0.13919891372058496), (0.2147606538674065, -0.056807480758377715, -0.003371588472134381, 0.050802375009620836, -0.0966211768602331), (0.3069604237137369, 0.07246856451940384, 0.015177261181389557, 0.12372550939010493, -0.14170804426614492), (0.027268912238764154, 0.0017904850104751777, 0.0009033263540078095, 0.3954724763846459, 0.009679310061993546), (0.06876059227571307, -0.08890003868062943, -0.020022264952405895, 0.15842948283883918, -0.03451431894408288), (0.0998180012382972, -0.0026630790348873556, -0.005509269895703791, 0.16012597895615774, -0.04332808342112766), (0.0

In [38]:
features = xk_mim[:5]
features
data

,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,...,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,class
0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0
1,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0
2,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0
4,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4996,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4997,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4998,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1


In [39]:
# mim top five features
parameters = {'n_estimators': [200, 500], 'max_features': ['sqrt', 'log2'], 'max_depth': [4, 5, 6, 7, 8], 'criterion':['gini', 'entropy', 'log_loss']}
rfc=RandomForestClassifier(random_state=42)
clf = GridSearchCV(rfc, parameters, error_score='raise')
features = xk_mim[:5]
print(features)
x = data.iloc[:, features]
y = data['class']
print(x)
# print(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=1, stratify=y)
clf.fit(x_train, y_train)
sorted(clf.cv_results_)
print(clf.best_params_)
predictions = clf.predict(x_test)
print("Random Forest Accuracy")
print(accuracy_score(y_test, predictions))

[32, 5, 3, 13, 11]
      ct_state_ttl  sttl  sbytes  dpkts  dload
0              0.0   1.0     1.0    0.0    0.0
1              0.0   1.0     1.0    0.0    0.0
2              0.0   0.0     1.0    1.0    1.0
3              0.0   1.0     1.0    0.0    0.0
4              0.0   0.0     1.0    1.0    1.0
...            ...   ...     ...    ...    ...
4995           0.0   0.0     1.0    1.0    1.0
4996           0.0   0.0     1.0    1.0    1.0
4997           0.0   0.0     1.0    1.0    1.0
4998           1.0   1.0     0.0    0.0    0.0
4999           0.0   0.0     1.0    1.0    1.0

[5000 rows x 5 columns]
{'criterion': 'gini', 'max_depth': 4, 'max_features': 'sqrt', 'n_estimators': 200}
Random Forest Accuracy
0.978


In [40]:
# mrmr top five features
parameters = {'n_estimators': [200, 500], 'max_features': ['sqrt', 'log2'], 'max_depth': [4, 5, 6, 7, 8], 'criterion':['gini', 'entropy', 'log_loss']}
rfc=RandomForestClassifier(random_state=42)
clf = GridSearchCV(rfc, parameters, error_score='raise')
features = xk_mrmr[:5]
print(features)
x = data.iloc[:, features]
y = data['class']
print(x)
# print(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=1, stratify=y)
clf.fit(x_train, y_train)
sorted(clf.cv_results_)
print(clf.best_params_)
predictions = clf.predict(x_test)
print("Random Forest Accuracy")
print(accuracy_score(y_test, predictions))

[32, 41, 3, 0, 5]
      ct_state_ttl  ct_dst_sport_ltm  sbytes  proto  sttl
0              0.0               1.0     1.0    0.0   1.0
1              0.0               1.0     1.0    0.0   1.0
2              0.0               0.0     1.0    0.0   0.0
3              0.0               1.0     1.0    0.0   1.0
4              0.0               0.0     1.0    0.0   0.0
...            ...               ...     ...    ...   ...
4995           0.0               0.0     1.0    0.0   0.0
4996           0.0               0.0     1.0    0.0   0.0
4997           0.0               0.0     1.0    0.0   0.0
4998           1.0               1.0     0.0    0.0   1.0
4999           0.0               0.0     1.0    0.0   0.0

[5000 rows x 5 columns]
{'criterion': 'gini', 'max_depth': 4, 'max_features': 'sqrt', 'n_estimators': 500}
Random Forest Accuracy
0.977


In [41]:
# dimrmr top five features
parameters = {'n_estimators': [200, 500], 'max_features': ['sqrt', 'log2'], 'max_depth': [4, 5, 6, 7, 8], 'criterion':['gini', 'entropy', 'log_loss']}
rfc=RandomForestClassifier(random_state=42)
clf = GridSearchCV(rfc, parameters, error_score='raise')
features = xk_dimrmr[:5]
print(features)
x = data.iloc[:, features]
y = data['class']
print(x)
# print(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=1, stratify=y)
clf.fit(x_train, y_train)
sorted(clf.cv_results_)
print(clf.best_params_)
predictions = clf.predict(x_test)
print("Random Forest Accuracy")
print(accuracy_score(y_test, predictions))

[32, 9, 3, 0, 5]
      ct_state_ttl  service  sbytes  proto  sttl
0              0.0      0.0     1.0    0.0   1.0
1              0.0      0.0     1.0    0.0   1.0
2              0.0      1.0     1.0    0.0   0.0
3              0.0      0.0     1.0    0.0   1.0
4              0.0      0.0     1.0    0.0   0.0
...            ...      ...     ...    ...   ...
4995           0.0      1.0     1.0    0.0   0.0
4996           0.0      0.0     1.0    0.0   0.0
4997           0.0      1.0     1.0    0.0   0.0
4998           1.0      0.0     0.0    0.0   1.0
4999           0.0      1.0     1.0    0.0   0.0

[5000 rows x 5 columns]
{'criterion': 'gini', 'max_depth': 4, 'max_features': 'sqrt', 'n_estimators': 200}
Random Forest Accuracy
0.976
